# Import Package

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
user_test_folder = volatility_folder + "user_test/"
model_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/VWAP_TWAP/"

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import floor

import torch.nn as nn
import os.path as osp
import torch.utils.data as data
import torch.nn.functional as F

from tqdm import tqdm
import time, matplotlib, torch, os
from statsmodels.tsa.arima_model import ARIMA
from torch.utils.tensorboard import SummaryWriter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import os
from os import walk

import plotly
%matplotlib inline
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
#import seaborn as sns

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
#For DNN / CNN+LSTM
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Flatten, Dense

In [ ]:
!pip install -q stockstats

# Load Data

In [ ]:
import pandas_datareader.data as web
from stockstats import StockDataFrame

#Setting the index values as timestamp
def process(code):
  print("Enter the code of the model that you want to implement")
  #code = input()
  #final_code = '^'+code
  final_code = code
  start_date = "2012-01-01"
  end_date = "2021-12-31"
  stock = web.DataReader(final_code , 'stooq',start=start_date, end=end_date)
  stock.columns = ["open","high","low","close","volume"]
  
  # Use online package to generate additional features
  x = StockDataFrame(stock)
  stock = x[['open','high','low','close','volume',
          'boll', 'boll_ub', 'boll_lb',
          'macd', 'macdh', 'macds',
          'rsi_11', 'rsi_14', 'rsi_21']]
  stock.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in stock.index]
  stock = stock.sort_index()

  return stock

In [ ]:
stock = process("IBM")

Enter the code of the model that you want to implement


In [ ]:
stock

,open,high,low,close,volume,boll,boll_ub,boll_lb,macd,macdh,macds,rsi_11,rsi_14,rsi_21
20120103,128.88,130.23,128.38,128.58,8180058,128.1790,134.932612,121.425388,-1.811274,0.489102,-2.300376,51.281366,47.190995,42.920670
20120104,128.10,128.60,127.63,128.07,6297526,128.3965,135.476771,121.316229,-2.091832,0.330819,-2.422652,48.488535,44.808561,41.192557
20120105,127.56,127.71,126.38,127.46,6466200,128.6025,135.879775,121.325225,-2.367574,0.137783,-2.505356,45.064237,41.897279,39.091982
20120106,127.28,127.34,125.83,125.98,7094969,128.9115,136.502568,121.320432,-2.622973,-0.083171,-2.539802,35.625325,34.061158,33.612565
20120109,125.76,125.80,124.42,125.33,7535572,129.2665,136.943913,121.589087,-2.760113,-0.241104,-2.519009,30.883945,30.223364,31.016798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,129.08,130.08,128.40,130.05,4346229,131.4500,133.282321,129.617679,-0.099397,-0.058552,-0.040846,8.180361,8.545592,8.997984
20211228,130.03,131.63,130.02,131.05,3488601,131.8000,132.899939,130.700061,-0.042801,-0.026749,-0.016052,14.491914,14.911940,15.424720
20211229,130.82,132.61,130.73,131.76,4291007,132.0500,132.611070,131.488930,-0.010219,-0.007870,-0.002349,28.868360,29.305681,29.832936
20211230,132.16,132.77,131.74,132.32,3196152,132.1950,132.548553,131.841447,0.005609,0.002493,0.003116,100.000000,100.000000,100.000000


# Data preprocessing for regression models

 Split the train and test data - for IBM

In [ ]:
# Split the train and test data
def custom_split(data,start,end):
    train = (data.index >= start) & (data.index <= end)
    train_X = data[train]
    
    return train_X

In [ ]:
train_X = custom_split(stock,start = 20130101,end = 20171031)
valid_X = custom_split(stock,start = 20171101,end = 20181231)
test_X = custom_split(stock,start = 20190101,end = 20201231)

In [ ]:
print(len(train_X))
print(len(valid_X))
print(len(test_X))

1218
292
505


Label the target result (opening price on 11th day)

In [ ]:
# Assume we use 10 days price data to predict closing price of the 11th day
num_day_to_predict = 5

In [ ]:
def produce_result_target_price(X,num_day = 5,result_col_name = "result_price"):
    y = pd.DataFrame(np.nan, index=X.index, columns=[result_col_name])
    for i in range(5,len(X)):
        y.loc[X.index[i-1],result_col_name] = X.loc[X.index[i],"open"]
    
    return y

In [ ]:
train_y = produce_result_target_price(train_X,num_day_to_predict)
valid_y = produce_result_target_price(valid_X,num_day_to_predict)
test_y = produce_result_target_price(test_X,num_day_to_predict)

In [ ]:
print(len(train_y))
print(len(valid_y))
print(len(test_y))

1218
292
505


In [ ]:
pred_y = produce_result_target_price(stock,num_day_to_predict)

Transform the 10-day data into one vector

In [ ]:
def transform_x_data_to_one_vector(X,num_day):
    col_name = []
    for i in range(num_day):
        for j in X.columns:
            col_name.append(j+"-"+str(i))
    new_X = pd.DataFrame(np.nan, index=X.index, columns=col_name)
    
    for i in range(num_day-1,len(X)):
        for col in col_name:
            split_list = col.split("-")
            new_X.loc[X.index[i],col] = X.loc[X.index[i-int(split_list[1])],split_list[0]]
    
    return new_X

In [ ]:
train_X_5 = transform_x_data_to_one_vector(train_X,num_day_to_predict)
valid_X_5 = transform_x_data_to_one_vector(valid_X,num_day_to_predict)
test_X_5 = transform_x_data_to_one_vector(test_X,num_day_to_predict)

In [ ]:
pred_X_5 = transform_x_data_to_one_vector(stock,num_day_to_predict)

In [ ]:
print(len(train_X_5))
print(len(valid_X_5))
print(len(test_X_5))
print(len(pred_X_5))

1218
292
505
2517


 Drop out rows with NaN

In [ ]:
def drop_nan_row_y(y,num_day = 5):
    drop_list = [y.index[i] for i in range(num_day-1)]
    drop_list.append(y.index[-1])
    return y.drop(drop_list)

def drop_nan_row_X_5(X,num_day = 5):
    drop_list = [X.index[i] for i in range(num_day-1)]
    drop_list.append(X.index[-1])
    return X.drop(drop_list)

In [ ]:
new_train_X_5 = drop_nan_row_X_5(train_X_5,num_day_to_predict)
new_train_y = drop_nan_row_y(train_y,num_day_to_predict)

new_valid_X_5  = drop_nan_row_X_5(valid_X_5,num_day_to_predict)
new_valid_y = drop_nan_row_y(valid_y,num_day_to_predict)

new_test_X_5  = drop_nan_row_X_5(test_X_5,num_day_to_predict)
new_test_y = drop_nan_row_y(test_y,num_day_to_predict)

In [ ]:
new_pred_X_5  = drop_nan_row_X_5(pred_X_5,num_day_to_predict)
new_pred_y = drop_nan_row_y(pred_y,num_day_to_predict)

In [ ]:
print(len(new_train_X_5))
print(len(new_valid_X_5))
print(len(new_test_X_5))
print(len(new_pred_X_5))

1213
287
500
2512


In [ ]:
print(len(new_train_y))
print(len(new_valid_y))
print(len(new_test_y))
print(len(new_pred_y))

1213
287
500
2512


In [ ]:
# Keep a copy for reverting prediction result
old_train_X_5 = new_train_X_5
old_train_y = new_train_y
old_valid_X_5 = new_valid_X_5
old_valid_y = new_valid_y
old_test_X_5 = new_test_X_5
old_test_y = new_test_y

In [ ]:
old_pred_X_5 = new_pred_X_5
old_pred_y = new_pred_y

In [ ]:
### Normalize data row by row

In [ ]:
def normalize_data_by_row(X,y):
    norm_X = X.sub(X.mean(axis=0), axis=1).div(X.std(axis=0), axis=1)
    
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
    
    temp_y = y.loc[:,"result_price"]
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = temp_y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = temp_y.index
    

    norm_y = pd.DataFrame((temp_y-mean_5_day)/std_5_day,
                          columns=["result_price"])
                         
    return (norm_X,norm_y)

In [ ]:
new_train_X_5,new_train_y = normalize_data_by_row(new_train_X_5,new_train_y)
new_valid_X_5,new_valid_y = normalize_data_by_row(new_valid_X_5,new_valid_y)
new_test_X_5,new_test_y = normalize_data_by_row(new_test_X_5,new_test_y)

In [ ]:
new_pred_X_5,new_pred_y = normalize_data_by_row(new_pred_X_5,new_pred_y)

In [ ]:
new_train_X_5 = drop_nan_row_X_5(new_train_X_5,num_day_to_predict)
new_valid_X_5 = drop_nan_row_X_5(new_valid_X_5,num_day_to_predict)
new_test_X_5 = drop_nan_row_X_5(new_test_X_5,num_day_to_predict)

In [ ]:
new_train_y = drop_nan_row_y(new_train_y,num_day_to_predict)
new_valid_y = drop_nan_row_y(new_valid_y,num_day_to_predict)
new_test_y = drop_nan_row_y(new_test_y,num_day_to_predict)

In [ ]:
len(new_test_X_5.columns)

70

In [ ]:
len(new_train_X_5)
#len(new_valid_X_5)
#len(new_test_X_5)
#len(new_pred_X_5)

1208

In [ ]:
def convert_prediction(X,y,pred):

    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = y.index
    a = pd.DataFrame(index = y.index,columns=['mean','std','pred'])
    a.loc[:,"mean"] = mean_5_day
    a.loc[:,"std"] = std_5_day
    a.loc[:,"pred"] = pred
    a.loc[:,"converted"] = pred*std_5_day+mean_5_day
    return a["converted"]

In [ ]:
def convert_decision(stock_,pred):
    h = np.array(pred)
    action = []
    status = "N"
    five_sma = stock_['open'].rolling(window=5,min_periods=1).mean()

    ind_sma = five_sma.index
    ind_pred = pred.index
    union_ind = list(ind_sma.intersection(ind_pred))

    five_sma = five_sma.filter(items = union_ind)
    pred = pred.filter(items = union_ind)

    for i in range(len(union_ind)):

        if pred.iloc[i] >= five_sma.iloc[i]:
            if status == "N":
                action.append("Buy")
                status = "Buy"
            else:
                action.append("Hold")
        else:
            if status == "Buy":
                action.append("Sell")
                status = "N"
            else:
                action.append("Hold")

    result = (pd.DataFrame(action,
                         columns=["Action"], index = pred.index))
                
    return result

# TWAP

## process

In [ ]:
def twap(df):
  opens = df.filter(like="open")
  closes = df.filter(like="close")
  highs = df.filter(like="high")
  lows = df.filter(like="low")
  return pd.concat([opens,closes,highs,lows],axis=1)


In [ ]:
final_train = twap(new_train_X_5)
final_train.head()

,open-0,open-1,open-2,open-3,open-4,close-0,close-1,close-2,close-3,close-4,high-0,high-1,high-2,high-3,high-4,low-0,low-1,low-2,low-3,low-4
20130114,0.801829,0.850700,0.788361,0.838681,0.804244,0.785525,0.893255,0.799874,0.767687,0.797469,0.760783,0.855580,0.740043,0.770836,0.757528,0.804657,0.868912,0.775701,0.794957,0.791411
20130115,0.715955,0.800826,0.849662,0.787207,0.837732,0.780621,0.784580,0.892179,0.798716,0.766452,0.727228,0.759847,0.854563,0.738945,0.769790,0.725498,0.803646,0.867862,0.774575,0.793856
20130116,0.756030,0.714977,0.799804,0.848485,0.786275,0.784707,0.779677,0.783536,0.890988,0.797469,0.755054,0.726300,0.758857,0.853426,0.737910,0.781807,0.724510,0.802615,0.866701,0.773481
20130117,0.859896,0.755040,0.713982,0.798646,0.847534,0.846824,0.783763,0.778635,0.782385,0.889704,0.829532,0.754119,0.725319,0.757753,0.852349,0.890343,0.780802,0.723504,0.801479,0.865575
20130118,0.871346,0.858876,0.754032,0.712857,0.797710,0.895047,0.845863,0.782720,0.777485,0.781144,0.856540,0.828578,0.753131,0.724226,0.756711,0.921354,0.889308,0.779779,0.722397,0.800376


In [ ]:
final_valid = twap(new_valid_X_5)

In [ ]:
final_test = twap(new_test_X_5)

In [ ]:
final_pred = twap(new_pred_X_5)

In [ ]:
#df['TWAP'] = (df['Open']+df["Close"]+df["Low"]+df["High"]) / 4

for i in range(5):
  #df_ = final.filter(like=f"-{i}")
  final_train[f"TWAP_{i}"] = (final_train[f"open-{i}"]+final_train[f"open-{i}"]+final_train[f"open-{i}"]+final_train[f"open-{i}"])/4
  final_test[f"TWAP_{i}"] = (final_test[f"open-{i}"]+final_test[f"open-{i}"]+final_test[f"open-{i}"]+final_test[f"open-{i}"])/4
  final_valid[f"TWAP_{i}"] = (final_valid[f"open-{i}"]+final_valid[f"open-{i}"]+final_valid[f"open-{i}"]+final_valid[f"open-{i}"])/4
  final_pred[f"TWAP_{i}"] = (final_pred[f"open-{i}"]+final_pred[f"open-{i}"]+final_pred[f"open-{i}"]+final_pred[f"open-{i}"])/4



In [ ]:
new_train_X_5 = final_train
new_test_X_5 = final_test 
new_valid_X_5 = final_valid
new_pred_X_5 = final_pred

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.0001, # minimium change considered improvement
    patience=50, # how many epochs to wait before stopping
    restore_best_weights=True,
)

#train_X, test_X, train_y, test_y
model = keras.Sequential([
    layers.Dense(units=1024, input_shape=[len(new_train_X_5.columns)]),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(1)

])

model.compile(
    optimizer='adam',
    loss='mae',
)

history = model.fit(
    new_train_X_5, new_train_y,
    validation_data=(new_valid_X_5, new_valid_y),
    batch_size=256,
    epochs=200,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=False
)

In [ ]:
pred_y = model.predict(new_test_X_5)

In [ ]:
len(pred_y)

495

In [ ]:
#Performance evaluation
print('TWAP PERFORMANCE')
print('r2 score: '+str(r2_score(new_test_y, pred_y)))
print('RMSE : '+str(np.sqrt(mean_squared_error(new_test_y, pred_y))))
print("Mean Absolute Error : " + str(mean_absolute_error(new_test_y,pred_y)))

TWAP PERFORMANCE
r2 score: 0.5584552566318358
RMSE : 1.4968494612872458
Mean Absolute Error : 0.94321567199208


In [ ]:
import pickle

In [ ]:
cd //

/


In [ ]:
cd content

[Errno 2] No such file or directory: 'content'
/content


In [ ]:
with open("TWAP.pickle", "wb") as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://a5d5b6be-9410-479f-99bc-51455e557119/assets
